In [1]:
import requests
from urllib.parse import urlparse, quote
import pandas as pd

In [3]:
key_fd = open('roadapikey.txt', mode='r')
roadapi_key = key_fd.read(100)
key_fd.close()

In [9]:
bldg_list = ['대구시청', '강원도청', '경북도청']
road_addr_list = []
for bldg in bldg_list:
    url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote(bldg)}&resultType=json'
    results = requests.get(url).json()
    addr_list = results['results']['juso']
    road_addr = addr_list[0]['roadAddr']
    road_addr_list.append(road_addr)
    print(road_addr)

대구광역시 중구 동덕로 182 (동인동1가)
강원도 춘천시 봉의산길 37 (봉의동)
경상북도 예천군 호명면 도청대로 53


In [11]:
import pandas as pd 
df = pd.DataFrame({'place':bldg_list, 'addr':road_addr_list})
df

,place,addr
0,대구시청,대구광역시 중구 동덕로 182 (동인동1가)
1,강원도청,강원도 춘천시 봉의산길 37 (봉의동)
2,경북도청,경상북도 예천군 호명면 도청대로 53


In [10]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [12]:
lat_list = []; lng_list = []
for i in df.index:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(df['addr'][i])
    result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [13]:
df['lat'] = lat_list
df['lng'] = lng_list
df

,place,addr,lat,lng
0,대구시청,대구광역시 중구 동덕로 182 (동인동1가),35.870827,128.604606
1,강원도청,강원도 춘천시 봉의산길 37 (봉의동),37.886141,127.732251
2,경북도청,경상북도 예천군 호명면 도청대로 53,36.575713,128.471807


In [24]:
df.lat.mean(), df.lng.mean()

(36.7775604471858, 128.26955488398)

In [14]:
import folium

In [22]:
map = folium.Map(location=[df.lat.mean(), df.lng.mean()], zoom_start=8)
folium.Marker(
    location=[36.575713, 128.471807],
    popup='경북도청'
).add_to(map)
map

In [23]:
map = folium.Map(location=[df.lat.mean(), df.lng.mean()], zoom_start=8)
for i in df.index:
    folium.Marker(
        location=[df.lat[i], df.lng[i]],
        popup=df.place[i],
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px"><b>대구시청, 강원도청, 경북도청</b></h3>'   
map.get_root().html.add_child(folium.Element(title_html))
map